In [1]:
import os

In [2]:
%pwd

'/Users/songfxu/Documents/study/MLOps-End2End/research'

In [3]:
os.chdir('../')
%pwd

'/Users/songfxu/Documents/study/MLOps-End2End'

In [4]:
import pandas as pd
data = pd.read_csv('artifacts/data_ingestion/winequality-red.csv')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [7]:
from pydantic import BaseModel
from pathlib import Path

class DataValidationConfig(BaseModel):
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [17]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

class DataValidationMgr():
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )
        return data_validation_config

In [13]:
import os
from src.datascience import logger

# component - data validation
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
    
    def validation_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    break

            if validation_status != False:
                validation_status = True
                
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f'Validation status: {validation_status}')
            return validation_status
        except Exception as e:
            raise e

In [19]:
try:
    mgr = DataValidationMgr()
    config = mgr.get_data_validation_config()
    entity = DataValidation(config=config)
    entity.validation_all_columns()
except Exception as e: 
    raise e

[2025-11-14 16:43:09,000: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-11-14 16:43:09,002: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-14 16:43:09,003: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-14 16:43:09,004: INFO: common: Created directory at: artifacts]
[2025-11-14 16:43:09,005: INFO: common: Created directory at: artifacts/data_validation]
